<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#파이프라인 테스트


In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
import math



In [2]:
!pip show transformers


Name: transformers
Version: 4.46.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [3]:

# 요약 모델 및 토크나이저 설정
summarizer_model_name = "EXP442/pegasus_summarizer"
summarizer_tokenizer = PegasusTokenizer.from_pretrained(summarizer_model_name)
summarizer_model = PegasusForConditionalGeneration.from_pretrained(summarizer_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

In [4]:
# 번역 모델 및 토크나이저 설정
translator_model_name = "EXP442/nllb_translator_pretrained"
model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name, forced_bos_token_id=256098)
tokenizer = AutoTokenizer.from_pretrained(translator_model_name, src_lang='eng_Latn', tgt_lang='kor_Hang')


config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.1k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

In [10]:
def translate_text(text):
    # 입력 텍스트를 토큰화하고 번역 수행
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    translated_ids = model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    # 번역 결과를 디코딩하고 불필요한 토큰 제거
    translation = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translation


In [11]:
# print(translate_text("2, 4, 6 etc. are even numbers."))

In [12]:
def split_text_with_last_sentence_overlap(text, target_chunk_length=2048):
    # 문장 단위로 텍스트 분할
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # 현재 문장을 청크에 추가
        if len(current_chunk) + len(sentence) <= target_chunk_length:
            current_chunk += sentence + " "
        else:
            # 청크에 문장 추가 후, 마지막 문장을 다음 청크에 포함
            chunks.append(current_chunk.strip())
            current_chunk = chunks[-1].split()[-1] + " " + sentence + " "

    # 마지막 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [13]:
def summarize_long_text(article_text, target_chunk_length=2048):
    # 텍스트를 분할하여 요약
    chunks = split_text_with_last_sentence_overlap(article_text, target_chunk_length)
    summaries = []

    for chunk in chunks:
        inputs = summarizer_tokenizer(chunk, max_length=target_chunk_length, return_tensors="pt", truncation=True)
        summary_ids = summarizer_model.generate(inputs["input_ids"], max_length=100, min_length=50, length_penalty=2.0, num_beams=2, early_stopping=True)
        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

In [14]:
def translate_and_combine_summaries(summaries):
    translated_summaries = [translate_text(summary) for summary in summaries]
    combined_translation = "\n".join(translated_summaries)
    return combined_translation

#번역할 기사

In [15]:
article_text = """
Japanese voters are today heading to the polls in a snap election, following a tumultuous few years for the ruling party which saw a “cascade” of scandals, widespread voter apathy and record-low approval ratings.

The election was announced by Shigeru Ishiba three days after he was selected as the leader of the ruling Liberal Democratic Party (LDP) - before he had been officially sworn in as prime minister.

The decision was made despite the LDP seeing approval ratings of below 20% earlier in the year, in the wake of a political fundraising corruption scandal.

Yet the LDP still remains the strongest contender against opposition parties which have failed to unite, or convince voters they are a viable option to govern.

The main opposition party had an approval rating of just 6.6% before parliament was dissolved.

“It is so hard to make decisions to choose parties, I think people are losing interest,” Miyuki Fujisaki, a long-time LDP supporter who works in the care-home sector, told the BBC ahead of polls opening.

The LDP, she said, has its problems with alleged corruption, “but the opposition also does not stand out at all”.

“They sure complain a lot, but it’s not at all clear on what they want to do,” the 66-year-old said.

For all the apathy, politics in Japan has been moving at a fast pace in recent months.

Shigeru Ishiba took over as prime minister after being voted in by the ruling party following his predecessor Fumio Kishida - who had been in the role since 2021 – making a surprise decision to step down in August.

The move to call the election came at a time when the LDP is desperate to restore its tarnished image among the public.


Ishiba - a long-time politician who previously served as defence minister - has described it as the “people’s verdict”.

But whether it’s enough to restore trust in the LDP - which has been in power almost continuously since 1955 – is uncertain.

A series of scandals has tarnished the ruling party’s reputation. Chief among them is the party’s relationship with the controversial Unification Church - described by critics as a “cult” - and the level of influence it had on lawmakers.

Then came the revelations of the political funding corruption scandal. Japan’s prosecutors have been investigating dozens of LDP lawmakers accused of pocketing proceeds from political fundraising events. Those allegations - running into the millions of dollars - led to the dissolution of powerful factions, the backbone of its internal party politics.

“What a wretched state the ruling party is in,” said Michiko Hamada, who had travelled to Urawa station, on outskirts of Tokyo, for an opposition campaign rally.

“That is what I feel most. It is tax evasion and it’s unforgivable.”

It strikes her as particularly egregious at a time when people in Japan are struggling with high prices. Wages have not changed for three decades – dubbed “the lost 30 years” – but prices have risen at the fastest rate in nearly half a century in the last two years.

This month, as voters were getting ready to go to the polls, saw more price hikes on thousands of food products as well as other day-to-day provisions like mail, pharmaceuticals, electricity and gas.

“I pay 10,000 yen or 20,000 yen ($65 - $130; £50 - £100) more for the food per month (than I used to),” Ms Hamada said.

“And I’m not buying the things I used to buy. I am trying to save up but it still costs more.



Things like fruit are very expensive.”

She is not the only one concerned with high prices. Pensioner Chie Shimizu says she now must work part-time to make ends meet.

“Our hourly wage has gone up a bit but it does not match the prices,” she told the BBC as she picked up some food from a stand at Urawa station. “I come to places like this to find something cheaper and good because everything in regular shops is expensive.”

Ms Shimizu has not voted for years but might this time - although she is not sure which candidate or which party to vote for.

“I can't find anyone that I want to vote for. I feel like there's no one who I can trust to be our leader. I wonder about those who become an MP for their own greed.”

Against this backdrop, it might look like Ishiba has taken a political gamble. His party had held 247 of 465 seats in the lower house, while its coalition partner, Komeito, had 32. A party needs 233 seats to control the house, known as the Diet.

There are now fears Komeito may fail to reach that number again, while the main opposition - which had 98 seats in the previous parliament - began to pick up momentum with voters ahead of Sunday’s election.

“I think the LDP has dug itself a very deep hole to climb out of. It does not enjoy public trust, and why should it?


There's just been a cascade of scandals,” Jeff Kingston, professor of Asian studies and history at Temple University Japan, told the BBC.

But he does not think any of this will necessarily lead to the party losing the elections.

“I think they (LDP) are worried they're going to lose some marginal seats, and there are questions hanging over Komeito and how effective a coalition partner they will be,” Prof Kingston said.

Should they pull off a win, Miyuki Fujisaki, the care-home sector worker, warns they will have to do more than just pay lip service to change.

“I want them to show us what they are going to do so this [the scandals] doesn’t happen again,” she said. “They need to prove it - not just say it like they often do in the election time.”


"""

#실행 코드


In [16]:
!pip install fastapi uvicorn pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.5 MB/s eta 0:00:00


In [17]:
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import threading

In [20]:
app = FastAPI()
ngrok.set_auth_token("2opqNgdgGSmrMGobhXVRaFQit1X_7Tc6i7X4T7fTtbJjqwuX")
# 요청 데이터 모델 정의
class TextData(BaseModel):
    text: str
@app.post("/process_text")
async def process_text(data: TextData):
    article_text = data.text

    # 텍스트 요약
    summaries = summarize_long_text(article_text)

    # 요약된 텍스트를 번역
    combined_translation = translate_and_combine_summaries(summaries)

    # 결과 반환
    return {"summary_translation": combined_translation}
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_app)
thread.start()

INFO:     Started server process [1523]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [21]:
# ngrok을 통해 FastAPI 앱을 노출
public_url = ngrok.connect(8000)
print("Public URL:", public_url)  # 생성된 공개 URL을 출력


Public URL: NgrokTunnel: "https://7efc-34-125-135-196.ngrok-free.app" -> "http://localhost:8000"


In [22]:
import requests

# 실제 공개 URL을 가져오기
url = public_url.public_url + "/process_text"
#article_text = "This is a sample text for testing the FastAPI process_text endpoint."

# 요청 보내기
response = requests.post(url, json={"text": article_text})

# 결과 출력
print("Response:", response.json())


INFO:     34.125.135.196:0 - "POST /process_text HTTP/1.1" 200 OK
Response: {'summary_translation': '일본 유권자들이 이날 임시선거에서 투표에 나선 가운데 집권 자민당(자민당)의 지지율이 사상 최저 수준으로 떨어졌고, 주요 야당은 의회가 해산되기 전까지 지지율이 6.6%에 불과했다.\n일본 여당은 최근 수개월간 잇따라 파문을 일으켰는데, 가장 큰 파문 중 하나는 정치자금 비리 파문이었지만, 여당은 여전히 집권하고 있으며 일요일 선거에서 승리할 것으로 예상된다. 임금은 30년 동안 변하지 않았지만 물가는 거의 반세기 만에 가장 빠른 속도로 상승했다.\n일본 여당은 최근 수개월간 잇따른 스캔들에 시달리며 일요일 선거를 앞두고 자민당이 하원 의석 465석 중 247석을 차지했고, 연립 파트너인 코메이토가 32석을 차지하는 등 주요 야당이 탄력을 받고 있다.'}


In [23]:
from pyngrok import ngrok
ngrok.kill()  # 모든 ngrok 터널 종료
public_url = ngrok.connect(8000)  # 새로 연결
print("Public URL:", public_url)

ERROR:pyngrok.process.ngrok:t=2024-11-14T09:54:47+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-14T09:54:47+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [37]:
combined_translation = translate_and_combine_summaries(summaries)
print(combined_translation)

NameError: name 'summaries' is not defined

In [ ]:
import time

In [ ]:
def process_text(article_text):
    # 총 시간 측정 시작
    total_start_time = time.time()

    # 요약 시간 측정
    start_time = time.time()
    summaries = summarize_long_text(article_text)
    end_time = time.time()
    summary_time = end_time - start_time
    print(f"Summarization time: {summary_time:.2f} seconds")

    # 번역 시간 측정
    start_time = time.time()
    combined_translation = translate_and_combine_summaries(summaries)
    end_time = time.time()
    translation_time = end_time - start_time
    print(f"Translation time: {translation_time:.2f} seconds")

    # 총 소요 시간 계산
    total_end_time = time.time()
    total_time = total_end_time - total_start_time
    print(f"Total processing time: {total_time:.2f} seconds")

    return combined_translation

In [ ]:
result = process_text(article_text)
print("Summary and Translation Result:\n", result)

Summarization time: 16.89 seconds
Translation time: 26.88 seconds
Total processing time: 43.78 seconds
Summary and Translation Result:
 일본 유권자들이 이날 임시선거에서 투표에 나선 가운데 집권 자민당(자민당)의 지지율이 사상 최저 수준으로 떨어졌고, 주요 야당은 의회가 해산되기 전까지 지지율이 6.6%에 불과했다.
일본 여당은 최근 수개월간 잇따라 파문을 일으켰는데, 가장 큰 파문 중 하나는 정치자금 비리 파문이었지만, 여당은 여전히 집권하고 있으며 일요일 선거에서 승리할 것으로 예상된다. 임금은 30년 동안 변하지 않았지만 물가는 거의 반세기 만에 가장 빠른 속도로 상승했다.
일본 여당은 최근 수개월간 잇따른 스캔들에 시달리며 일요일 선거를 앞두고 자민당이 하원 의석 465석 중 247석을 차지했고, 연립 파트너인 코메이토가 32석을 차지하는 등 주요 야당이 탄력을 받고 있다.


#동의어 사전 만들기 코드


In [ ]:
# from transformers import AutoTokenizer
# from sentence_transformers import SentenceTransformer, util

# from transformers import PegasusTokenizer, PegasusForConditionalGeneration
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import re
# import math
# import json

In [ ]:


# # Pegasus 및 Helsinki 토크나이저 로드
# summarizer_model_name = "/content/drive/MyDrive/summarizer/model/model_part_2"
# summarizer_tokenizer = PegasusTokenizer.from_pretrained(summarizer_model_name)

# translator_model_name = "/content/drive/MyDrive/translator/model/model_part_3"
# translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name)

# # Sentence-BERT 모델 로드 (동의어 검색용)
# embedding_model = SentenceTransformer("paraphrase-MiniLM-L3-v2")

# # 예시 단어 목록: Pegasus에는 있으나 Helsinki에는 없는 단어 추출
# pegasus_vocab = set(summarizer_tokenizer.get_vocab().keys())
# helsinki_vocab = set(translator_tokenizer.get_vocab().keys())
# exclusive_pegasus_words = list(pegasus_vocab - helsinki_vocab)




In [ ]:
# pegasus_vocab = set(summarizer_tokenizer.get_vocab().keys())
# helsinki_vocab = set(translator_tokenizer.get_vocab().keys())

# # Pegasus에만 존재하는 단어 집합
# exclusive_pegasus_words = list(pegasus_vocab - helsinki_vocab)
# helsinki_words = list(helsinki_vocab)

# # Helsinki 단어에 포함된 유사어를 찾기 위한 임베딩 생성
# helsinki_embeddings = embedding_model.encode(helsinki_words, convert_to_tensor=True)


In [ ]:
# import re
# import time
# import torch
# from tqdm import tqdm
# from sentence_transformers import util

# # Helsinki 모델의 vocabulary에서 영어 단어만 추출 (영어 알파벳으로만 구성된 단어)
# helsinki_words = [word for word in helsinki_vocab if re.match("^[a-zA-Z]+$", word)]

# # Helsinki 모델의 영어 단어들에 대한 임베딩 생성 및 텐서화
# helsinki_embeddings = [embedding_model.encode(word, convert_to_tensor=True) for word in helsinki_words]
# helsinki_embeddings = torch.stack(helsinki_embeddings)  # 텐서 형태로 변환

# # 동의어 사전 생성 함수 (영어 단어만, 유사도 95% 이상)
# def create_synonym_dict(exclusive_words, helsinki_words, helsinki_embeddings, threshold):
#     synonym_dict = {}
#     start_time = time.time()

#     for idx, word in enumerate(tqdm(exclusive_words[:10], desc="동의어 사전 생성 진행 중")):  # 10개만 테스트
#         # Pegasus 단어에 대한 임베딩 생성
#         print("현재 단어",word)
#         word_embedding = embedding_model.encode(word, convert_to_tensor=True)

#         # 코사인 유사도를 계산하여 유사한 Helsinki 단어 찾기
#         similarities = util.cos_sim(word_embedding, helsinki_embeddings)[0]
#         similar_words = [helsinki_words[i] for i, sim in enumerate(similarities) if sim >= threshold]

#         # 유사도가 95% 이상인 단어가 있으면 사전에 추가
#         if similar_words:
#             print("비슷한 단어",similar_words)
#             synonym_dict[word] = similar_words

#         # 남은 시간 계산
#         elapsed_time = time.time() - start_time
#         avg_time_per_word = elapsed_time / (idx + 1)
#         remaining_time = avg_time_per_word * (len(exclusive_words) - idx - 1)

#         print(f"남은 예상 시간: {remaining_time:.2f} 초", end="\n")

#     return synonym_dict

# # Pegasus에만 있는 단어들로 동의어 사전 생성 (10개만 테스트)
# synonym_dict = create_synonym_dict(exclusive_pegasus_words, helsinki_words, helsinki_embeddings, threshold=0.5)
# print("\n동의어 사전 생성 완료:", synonym_dict)


In [ ]:
# import json

# # 동의어 사전 JSON 파일로 저장
# with open("synonym_dict.json", "w") as f:
#     json.dump(synonym_dict, f, indent=2)

# print("동의어 사전이 'synonym_dict.json' 파일로 저장되었습니다.")


In [ ]:
# synonym_dict

#토크나이징 테스트


In [ ]:
# import json

# # 동의어 사전 로드
# def load_synonym_dict(file_path="synonym_dict.json"):
#     with open(file_path, "r") as f:
#         synonym_dict = json.load(f)
#     return synonym_dict

# # Pegasus 요약문의 단어를 동의어 사전에 따라 대체
# def replace_with_synonyms(text, synonym_dict):
#     tokens = text.split()  # 요약문을 단어 단위로 분리
#     modified_text = []

#     for token in tokens:
#         # 동의어 사전에 단어가 있으면 대체, 없으면 그대로 사용
#         if token in synonym_dict:
#             modified_text.append(synonym_dict[token][0])  # 첫 번째 동의어 사용
#         else:
#             modified_text.append(token)

#     # 수정된 텍스트를 다시 문장으로 결합
#     return " ".join(modified_text)

# # 동의어 사전 로드
# synonym_dict = load_synonym_dict("synonym_dict.json")



# # 요약문 변환
# for summarization in summaries:
#     processed_text = replace_with_synonyms(summarization, synonym_dict)
#     print("원본",summarization)
#     print("변환",processed_text)



#모델 hugging face 업로드

In [ ]:
# !pip install transformers huggingface_hub

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()  # API 토큰을 입력하면 됩니다.


In [ ]:
# save_directory = "./nllb_translator_pretrained"
# model.save_pretrained(save_directory)
# tokenizer.save_pretrained(save_directory)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'forced_bos_token_id': 256098}


('./nllb_translator_pretrained/tokenizer_config.json',
 './nllb_translator_pretrained/special_tokens_map.json',
 './nllb_translator_pretrained/sentencepiece.bpe.model',
 './nllb_translator_pretrained/added_tokens.json',
 './nllb_translator_pretrained/tokenizer.json')

In [ ]:
# from huggingface_hub import HfApi, HfFolder, Repository

# translator_model_repo = "EXP442/nllb_translator_pretrained"

# api = HfApi()

# # 모델을 업로드할 저장소 이름
# translator_model_repo = "EXP442/nllb_translator_pretrained"
# folder_path = "./nllb_translator_pretrained"  # 모델이 저장된 로컬 경로

# # Translator 모델 업로드
# api.upload_folder(folder_path=folder_path, repo_id=translator_model_repo)
# # Translator 모델 업로드
# # HfApi.upload_folder(folder_path=translator_model_name, repo_id=translator_model_repo)

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/EXP442/nllb_translator_pretrained/commit/cfc9332068985435eb45e7207bd3489ed7b46733', commit_message='Upload folder using huggingface_hub', commit_description='', oid='cfc9332068985435eb45e7207bd3489ed7b46733', pr_url=None, pr_revision=None, pr_num=None)